# Tutorial 101:

How to deploy a sequence task into the cluster using maestro lightning package.

In [1]:
import os, json
from maestro_lightning import Flow, Task, Dataset, Image

## 1) Task parameters:

In [2]:
basepath         = os.getcwd()
input_path       = f"{basepath}/jobs"
number_of_events = 2
number_of_jobs   = 2
run_number       = 20251211
image_path       = '/mnt/shared/storage03/projects/cern/data/images/lorenzetti_latest.sif'
repo_build_path  = '/home/joao.pinto/git_repos/lorenzetti/build'
binds            = {"/mnt/shared/storage03" : "/mnt/shared/storage03"}

## 2) Create the job dataset:

In [4]:
os.makedirs(input_path, exist_ok=True)
for job_id in range(number_of_jobs):
    with open(f"{input_path}/job_{job_id}.json", 'w') as f:
        nov = int(number_of_events / number_of_jobs)
        d = {
            'run_number'        : run_number,
            'seed'              : int(128 * (1+job_id)),
            'number_of_events'  : nov,
            #'events_per_job'    : int(nov/4),
            #'number_of_threads' : 4
        }
        json.dump(d,f)

## 3) Create the flow:

Let's create a flow of five sequence tasks from EVT to NTUPLE reconstruction.

In [5]:

with Flow(name="local_provider", path=f"{basepath}/local_tasks") as session:
    input_dataset    = Dataset(name="jobs", path=input_path)
    image            = Image(name="lorenzetti", path=image_path)


    pre_exec = f"source {repo_build_path}/lzt_setup.sh"

    command = f"{pre_exec} && gen_zee.py -o %OUT --job-file %IN -m"

    task_1 = Task(name="task_1.EVT",
                  image=image,
                  command=command,
                  input_data=input_dataset,
                  outputs={'OUT':'Zee.EVT.root'},
                  partition='cpu-large',
                  binds=binds)
    
    
    command = f"{pre_exec} && simu_trf.py -i %IN -o %OUT -nt 40"
    task_2 = Task(name="task_2.HIT",
                  image=image,
                  command=command,
                  input_data=task_1.output('OUT'),
                  outputs= {'OUT':'Zee.HIT.root'},
                  partition='cpu-large',
                  binds=binds)
    
    
    command = f"{pre_exec} && digit_trf.py -i %IN -o %OUT -nt 40"
    task_3 = Task(name="task_3.ESD",
                  image=image,
                  command=command,
                  input_data=task_2.output('OUT'),
                  outputs= {'OUT':'Zee.ESD.root'},
                  partition='cpu-large',
                  binds=binds)
    
    command = f"{pre_exec} && reco_trf.py -i %IN -o %OUT -nt 40"
    task_4 = Task(name="task_4.AOD",
                  image=image,
                  command=command,
                  input_data=task_3.output('OUT'),
                  outputs= {'OUT':'Zee.AOD.root'},
                  partition='cpu-large',
                  binds=binds)
    
    command = f"{pre_exec} && ntuple_trf.py -i %IN -o %OUT -nt 40"
    task_5 = Task(name="task_5.NTUPLE",
                  image=image,
                  command=command,
                  input_data=task_4.output('OUT'),
                  outputs= {'OUT':'Zee.NTUPLE.root'},
                  partition='cpu-large',
                  binds=binds)
   
   
    session.run()

15-Dec-2025 19:56:10 | Task task_1.EVT: creating output dataset 'Zee.EVT.root'.
15-Dec-2025 19:56:10 | Task task_2.HIT: looking for input dataset 'task_1.EVT.Zee.EVT.root'.
15-Dec-2025 19:56:10 | Task task_2.HIT: creating output dataset 'Zee.HIT.root'.
15-Dec-2025 19:56:10 | Task task_3.ESD: looking for input dataset 'task_2.HIT.Zee.HIT.root'.
15-Dec-2025 19:56:10 | Task task_3.ESD: creating output dataset 'Zee.ESD.root'.
15-Dec-2025 19:56:10 | Task task_4.AOD: looking for input dataset 'task_3.ESD.Zee.ESD.root'.
15-Dec-2025 19:56:10 | Task task_4.AOD: creating output dataset 'Zee.AOD.root'.
15-Dec-2025 19:56:10 | Task task_5.NTUPLE: looking for input dataset 'task_4.AOD.Zee.AOD.root'.
15-Dec-2025 19:56:10 | Task task_5.NTUPLE: creating output dataset 'Zee.NTUPLE.root'.
15-Dec-2025 19:56:10 | Running flow at /home/joao.pinto/git_repos/maestro-lightning/docs/local_tasks
15-Dec-2025 19:56:10 | No existing tasks found, initializing new flow.
15-Dec-2025 19:56:10 | Creating flow directory 

**NOTE**: Dont remove or modify any content inside of the `local_tasks` folder.

## 4) How to print the list of tasks?

In [6]:
!maestro task list -h

Usage: maestro list [-h] -i INPUT_FILE [--message-level MESSAGE_LEVEL]

Optional Arguments:
  -h, --help            show this help message and exit
  -i, --input INPUT_FILE
                        The job input file
  --message-level MESSAGE_LEVEL
                        Set the logging level (DEBUG, INFO, WARNING, ERROR,
                        CRITICAL). Default is INFO.


In [29]:
!maestro task list -i local_tasks

+---------------+------------+-----------+-----------+-------------+----------+-------------+-----------+
| taskname      |   assigned |   pending |   running |   completed |   failed |   finalized | status    |
|---------------+------------+-----------+-----------+-------------+----------+-------------+-----------|
| task_1.EVT    |          0 |         0 |         0 |           2 |        0 |           0 | completed |
| task_2.HIT    |          0 |         0 |         0 |           2 |        0 |           0 | completed |
| task_3.ESD    |          0 |         0 |         0 |           2 |        0 |           0 | completed |
| task_4.AOD    |          0 |         0 |         0 |           2 |        0 |           0 | completed |
| task_5.NTUPLE |          0 |         0 |         0 |           2 |        0 |           0 | completed |
+---------------+------------+-----------+-----------+-------------+----------+-------------+-----------+


## 5) How to retry?

In case of panic, use the retry method to recover all failed jobs.

In [8]:
!maestro task retry -h

Usage: maestro retry [-h] -i INPUT_FILE [--message-level MESSAGE_LEVEL]
                     [--dry-run]

Optional Arguments:
  -h, --help            show this help message and exit
  -i, --input INPUT_FILE
                        The job input file
  --message-level MESSAGE_LEVEL
                        Set the logging level (DEBUG, INFO, WARNING, ERROR,
                        CRITICAL). Default is INFO.
  --dry-run             Perform a dry run without executing the task.
